In [6]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def numericalTermOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct


def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct

In [ ]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()# output snipped 

import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
# output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

121
[0, 24*x^3 + 32*x]


In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

24*x^3 + 32*x
276*x^6 - 32*x^4 - 192*x^2
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
49152*x^15 - 1072627712/1125*x^13 + 4173856768/675*x^11 - 45736787968/3375*x^9 + 1564475392/675*x^7 + 6257901568/1125*x^5
184024*x^18 - 144266176/27*x^16 + 43337949824/729*x^14 - 217405085696/729*x^12 + 140124878848/243*x^10 - 77900890112/729*x^8 - 155801780224/729*x^6


In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print polynomialnumerator(D(poly))

24*x^3 + 32*x
276*x^6 - 32*x^4 - 192*x^2
55296*x^9 - 237568*x^7 + 32768*x^5 + 131072*x^3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
165888000*x^15 - 3217883136*x^13 + 20869283840*x^11 - 45736787968*x^9 + 7822376960*x^7 + 18773704704*x^5
134153496*x^18 - 3895186752*x^16 + 43337949824*x^14 - 217405085696*x^12 + 420374636544*x^10 - 77900890112*x^8 - 155801780224*x^6


In [8]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
for k in [1..20]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
print numericalTermOverField(poly,prime,sd)

(4) * (x + 1)^8 * (x + 2)^8 * (x + 3)^8 * (x + 4)^8 * x^20
4


In [9]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        nt=numericalTermOverFieldNoPrint(poly,prime,sd)
       
        data=data+[nt]
print data

[4, 1, 1, 4, 3, 2, 4, 4, 1, 4, 4, 1, 1, 4, 4, 1, 2, 4, 3, 3, 3, 4, 1, 3]


In [22]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=12
        nt=numericalTermOverFieldNoPrint(poly,prime,sd)
       
        data=data+[nt]
print data

[4, 1, 1, 4, 3, 2, 4, 4, 1, 4, 4, 1, 1, 4, 4, 1, 2, 4, 3, 3, 3, 4, 1, 3]


In [21]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        nt=numericalTermOverFieldNoPrint(poly,prime,sd)
        if nt==1:
            data=data+[index/5]
print data

[1, 2, 8, 11, 12, 15, 22]


In [18]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    data=data+[nt]
print data

[4, 1, 1, 2, 4, 1, 1, 2, 4, 1, 1, 2, 2, 4, 3, 4, 1, 4, 3, 4, 3, 4, 2, 3, 3, 2, 3, 2, 2, 4, 4, 1, 4, 3, 2, 4, 4, 4, 1, 2, 1, 2, 2, 4, 4, 4, 2, 4, 1, 2, 4, 1, 1, 1, 3, 1, 3, 4, 4, 1, 1, 4, 2, 3, 2, 4, 1, 3, 4, 2, 4, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 1, 1, 2, 4, 4, 4, 3, 1, 4, 3, 2, 2, 3, 3, 3, 2, 3, 4, 1, 3, 3, 3, 1, 1, 4, 4, 2, 2, 4, 1, 2, 3, 2, 4, 3, 2, 1, 3, 4]


In [23]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=12
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    data=data+[nt]
print data

[4, 1, 1, 2, 4, 1, 1, 2, 4, 1, 1, 2, 2, 4, 3, 4, 1, 4, 3, 4, 3, 4, 2, 3, 3, 2, 3, 2, 2, 4, 4, 1, 4, 3, 2, 4, 4, 4, 1, 2, 1, 2, 2, 4, 4, 4, 2, 4, 1, 2, 4, 1, 1, 1, 3, 1, 3, 4, 4, 1, 1, 4, 2, 3, 2, 4, 1, 3, 4, 2, 4, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 1, 1, 2, 4, 4, 4, 3, 1, 4, 3, 2, 2, 3, 3, 3, 2, 3, 4, 1, 3, 3, 3, 1, 1, 4, 4, 2, 2, 4, 1, 2, 3, 2, 4, 3, 2, 1, 3, 4]


In [24]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=12
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    if nt==1:data=data+[index]
print data

[1, 2, 5, 6, 9, 10, 16, 31, 38, 40, 48, 51, 52, 53, 55, 59, 60, 66, 74, 75, 79, 81, 82, 88, 99, 103, 104, 110, 117]


In [27]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=12
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    if nt==1:
        print index.digits(prime)

[1]
[2]
[0, 1]
[1, 1]
[4, 1]
[0, 2]
[1, 3]
[1, 1, 1]
[3, 2, 1]
[0, 3, 1]
[3, 4, 1]
[1, 0, 2]
[2, 0, 2]
[3, 0, 2]
[0, 1, 2]
[4, 1, 2]
[0, 2, 2]
[1, 3, 2]
[4, 4, 2]
[0, 0, 3]
[4, 0, 3]
[1, 1, 3]
[2, 1, 3]
[3, 2, 3]
[4, 4, 3]
[3, 0, 4]
[4, 0, 4]
[0, 2, 4]
[2, 3, 4]


In [25]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=12
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    if nt==1:data=data+[mod(index,5)]
print data

[1, 2, 0, 1, 4, 0, 1, 1, 3, 0, 3, 1, 2, 3, 0, 4, 0, 1, 4, 0, 4, 1, 2, 3, 4, 3, 4, 0, 2]


In [19]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    if nt==1:data=data+[index]
print data

[1, 2, 5, 6, 9, 10, 16, 31, 38, 40, 48, 51, 52, 53, 55, 59, 60, 66, 74, 75, 79, 81, 82, 88, 99, 103, 104, 110, 117]


In [10]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
for residue in [0..4]:
    data=[]
    for k in [1..len(s)-1]:
        index=s[k][0]
        if mod(index,prime)==residue:
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            data=data+[nt]
    print data
    print

[4, 1, 1, 4, 3, 2, 4, 4, 1, 4, 4, 1, 1, 4, 4, 1, 2, 4, 3, 3, 3, 4, 1, 3]

[1, 1, 2, 1, 4, 3, 1, 4, 2, 2, 1, 3, 4, 1, 2, 3, 1, 4, 2, 2, 3, 4, 2, 2]

[1, 2, 2, 4, 2, 2, 4, 4, 2, 4, 1, 4, 2, 3, 2, 3, 1, 3, 2, 3, 3, 2, 3, 1]

[2, 4, 4, 3, 3, 2, 3, 1, 4, 1, 1, 4, 3, 4, 3, 2, 2, 1, 3, 4, 1, 2, 2, 3]

[4, 1, 3, 4, 3, 4, 2, 2, 4, 2, 3, 1, 2, 2, 1, 1, 4, 4, 3, 1, 1, 4, 4, 4]



In [11]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120

prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print (index,sd,od,nt)


(5, 1, 1, 1)
(10, 1, 1, 1)
(15, 1, 1, 4)
(20, 1, 1, 3)
(25, 1, 2, 2)
(30, 1, 1, 4)
(35, 1, 1, 4)
(40, 1, 1, 1)
(45, 1, 1, 4)
(50, 1, 2, 4)
(55, 1, 1, 1)
(60, 1, 1, 1)
(65, 1, 1, 4)
(70, 1, 1, 4)
(75, 1, 2, 1)
(80, 1, 1, 2)
(85, 1, 1, 4)
(90, 1, 1, 3)
(95, 1, 1, 3)
(100, 1, 2, 3)
(105, 1, 1, 4)
(110, 1, 1, 1)
(115, 1, 1, 3)


In [12]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
data1=[]
data2=[]
data3=[]
data4=[]
digits1=[]
digits2=[]
digits3=[]
digits4=[]
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    dgt=index.digits(prime)
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            if nt==1:
                data1=data1+[index.factor()]
                digits1=digits1+[dgt]
            if nt==2:
                data2=data2+[index.factor()]
                digits2=digits2+[dgt]
            if nt==3:
                data3=data3+[index.factor()]
                digits3=digits3+[dgt]
            if nt==4:
                data4=data4+[index.factor()]
                digits4=digits4+[dgt]
            

for k in [0..len(data1)-1]:
    print (1,data1[k])
for k in [0..len(data2)-1]:
    print (2,data2[k])
for k in [0..len(data3)-1]:
    print (3,data1[k])
for k in [0..len(data4)-1]:
    print (4,data4[k])


(1, 5)
(1, 2 * 5)
(1, 2^3 * 5)
(1, 5 * 11)
(1, 2^2 * 3 * 5)
(1, 3 * 5^2)
(1, 2 * 5 * 11)
(2, 5^2)
(2, 2^4 * 5)
(3, 5)
(3, 2 * 5)
(3, 2^3 * 5)
(3, 5 * 11)
(3, 2^2 * 3 * 5)
(4, 3 * 5)
(4, 2 * 3 * 5)
(4, 5 * 7)
(4, 3^2 * 5)
(4, 2 * 5^2)
(4, 5 * 13)
(4, 2 * 5 * 7)
(4, 5 * 17)
(4, 3 * 5 * 7)


In [13]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print ((index-residue)/prime,od,nt)

(1, 1, 1)
(2, 1, 1)
(3, 1, 4)
(4, 1, 3)
(5, 2, 2)
(6, 1, 4)
(7, 1, 4)
(8, 1, 1)
(9, 1, 4)
(10, 2, 4)
(11, 1, 1)
(12, 1, 1)
(13, 1, 4)
(14, 1, 4)
(15, 2, 1)
(16, 1, 2)
(17, 1, 4)
(18, 1, 3)
(19, 1, 3)
(20, 2, 3)
(21, 1, 4)
(22, 1, 1)
(23, 1, 3)


In [14]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            print
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            nt=numericalTermOverField(poly,prime,sd)
           


(x + 2)^2 * (x + 3)^2 * (x + 1)^4 * (x + 4)^4 * x^6

(x + 2)^4 * (x + 3)^4 * (x + 1)^6 * (x + 4)^6 * x^11

(4) * (x + 2)^6 * (x + 3)^6 * (x + 1)^8 * (x + 4)^8 * x^16

(3) * (x + 2)^8 * (x + 3)^8 * (x + 1)^10 * (x + 4)^10 * x^21

(2) * (x + 2)^10 * (x + 3)^10 * (x + 1)^12 * (x + 4)^12 * x^26

(4) * (x + 2)^12 * (x + 3)^12 * (x + 1)^14 * (x + 4)^14 * x^31

(4) * (x + 2)^14 * (x + 3)^14 * (x + 1)^16 * (x + 4)^16 * x^36

(x + 2)^16 * (x + 3)^16 * (x + 1)^18 * (x + 4)^18 * x^41

(4) * (x + 2)^18 * (x + 3)^18 * (x + 1)^20 * (x + 4)^20 * x^46

(4) * (x + 2)^20 * (x + 3)^20 * (x + 1)^22 * (x + 4)^22 * x^51

(x + 2)^22 * (x + 3)^22 * (x + 1)^24 * (x + 4)^24 * x^56

(x + 2)^24 * (x + 3)^24 * (x + 1)^26 * (x + 4)^26 * x^61

(4) * (x + 2)^26 * (x + 3)^26 * (x + 1)^28 * (x + 4)^28 * x^66

(4) * (x + 2)^28 * (x + 3)^28 * (x + 1)^30 * (x + 4)^30 * x^71

(x + 2)^30 * (x + 3)^30 * (x + 1)^32 * (x + 4)^32 * x^76

(2) * (x + 2)^32 * (x + 3)^32 * (x + 1)^34 * (x + 4)^34 * x^81

(4) * (x + 2)^34 * (x + 3)

In [15]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            od=ord(prime,index-residue)
            poly = s[k][1]
            poly=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(poly,prime,bound)
            td=totalDegreeOverField(poly,prime,sd)
            nt=numericalTermOverFieldNoPrint(poly,prime,sd)
            print (index,td,nt)
           

(5, 18, 1)
(10, 31, 1)
(15, 44, 4)
(20, 57, 3)
(25, 70, 2)
(30, 83, 4)
(35, 96, 4)
(40, 109, 1)
(45, 122, 4)
(50, 135, 4)
(55, 148, 1)
(60, 161, 1)
(65, 174, 4)
(70, 187, 4)
(75, 200, 1)
(80, 213, 2)
(85, 226, 4)
(90, 239, 3)
(95, 252, 3)
(100, 265, 3)
(105, 278, 4)
(110, 291, 1)
(115, 304, 3)


In [17]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
sd=12
prime = 5
residue=0
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if index-residue!=0:
        od=ord(prime,index-residue)
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        td=totalDegreeOverField(poly,prime,sd)
        nt=numericalTermOverFieldNoPrint(poly,prime,sd)
        if td==nt:
            data=data+[index]
            print (index,nt)
print data

(1, 1)
(3, 2)
(7, 2)
(9, 1)
(10, 1)
(11, 2)
(15, 4)
(28, 2)
(51, 1)
(54, 3)
(59, 1)
(60, 1)
(63, 3)
(65, 4)
(67, 3)
(69, 2)
(78, 2)
(89, 4)
(98, 4)
(106, 4)
(107, 2)
(110, 1)
(111, 2)
(114, 4)
[1, 3, 7, 9, 10, 11, 15, 28, 51, 54, 59, 60, 63, 65, 67, 69, 78, 89, 98, 106, 107, 110, 111, 114]


In [32]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        od=ord(prime,index-residue)
        poly = D(s[k][1])
        pn=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(pn,prime,bound)
        nt=numericalTermOverFieldNoPrint(pn,prime,sd)
        nf=numericalfactor(poly)
        print (nf,mod(nf,prime),nt)
            
           

(276, 1, 1)
(2048, 3, 1)
(11202, 2, 2)
(49152, 2, 4)
(184024, 4, 1)
(614400, 0, 1)
(1881471, 1, 2)
(5373952, 2, 4)
(14478180, 0, 1)
(37122048, 3, 1)
(91231550, 0, 2)
(216072192, 2, 2)
(495248952, 2, 4)
(1102430208, 3, 3)
(2390434947, 2, 4)
(5061476352, 2, 1)
(10487167336, 1, 4)
(21301241856, 1, 3)
(42481784514, 4, 4)
(83300614144, 4, 3)
(160791890304, 4, 4)
(305854488576, 1, 2)
(573872089212, 2, 3)
(1063005978624, 4, 3)
(1945403602764, 4, 2)
(3519965179904, 4, 3)
(6300794030460, 0, 2)
(11164248047616, 1, 2)
(19591528119288, 3, 4)
(34065932304384, 4, 4)
(58718797964805, 0, 1)
(100372723007488, 3, 4)
(170215559855424, 4, 3)
(286470013685760, 0, 2)
(478625723149576, 1, 4)
(794110053826560, 0, 4)
(1308745319975256, 1, 4)
(2143055278039040, 0, 1)
(3487563372381816, 1, 2)
(5641848336678912, 2, 1)
(9074553043554568, 3, 2)
(14515166263443456, 1, 2)
(23093778743102262, 2, 4)
(36552977852071936, 1, 4)
(57567784186189368, 3, 4)
(90226777113575424, 4, 2)
(140752796480416011, 1, 4)
(218578429975461

In [33]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            poly = D(s[k][1])
            pn=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(pn,prime,bound)
            nt=numericalTermOverFieldNoPrint(pn,prime,sd)
            nf=numericalfactor(poly)
            print (mod(nf,prime),nt)
            
           

(4, 1)
(3, 1)
(2, 4)
(4, 3)
(4, 2)
(4, 4)
(1, 4)
(2, 1)
(3, 4)
(3, 4)
(3, 1)
(2, 1)
(2, 4)
(2, 4)
(2, 1)
(1, 2)
(1, 4)
(1, 3)
(0, 3)
(4, 3)
(3, 4)
(1, 1)
(2, 3)


In [35]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime = 5
residue=0
for k in [1..len(s)-1]:
    index=s[k][0]
    if index!=0:
        if mod(index,prime)==residue:
            poly = D(s[k][1])
            pn=polynomialnumerator(D(poly))
            sd=leastSplittingDegree(pn,prime,bound)
            nt=numericalTermOverFieldNoPrint(pn,prime,sd)
            nf=numericalfactor(poly)
            data=data+[mod(nf,prime)*nt]
print data

[4, 3, 3, 2, 3, 1, 4, 2, 2, 2, 3, 2, 3, 3, 2, 2, 4, 3, 0, 2, 2, 1, 1]
